In [1]:
import pandas as pd
import numpy as np

In [2]:
class MatchRating:
    original_rating_name = "Original"

    def __init__(self, player_file, match_file):
        self.players = pd.read_csv(player_file)
        self.players.set_index('ShortName', inplace=True)

        self.matches = pd.read_csv(match_file)

        self.__player_ratings = pd.DataFrame(columns=["Rating"], index=self.players.index)
        self.__player_ratings["Rating"] = 0.0
        

    def calculate_ratings(self, max_iterations, step_size):
        
        total_match_weights  = self.matches["K"].sum()

        for i in range(max_iterations):
            rating_table = self.__create_rating_table(step_size)
            
            match_score_accuracy = self.__calculate_match_score_accuracy(rating_table)        
            
            player_rating_accuracy = pow(match_score_accuracy.prod(), 1 / total_match_weights)
            accuracy_product = player_rating_accuracy.sort_values(ascending=False)

            best_accuracy = accuracy_product.index[0]
            print(f"Step Size={step_size}, Iteration {i + 1}, Accuracy: {round(accuracy_product.iloc[0], 5)}")

            if best_accuracy == MatchRating.original_rating_name:
                return self.__player_ratings.sort_values(by="Rating", ascending=False)
            
            player, rating_change = best_accuracy            
            self.__player_ratings.loc[player] += rating_change
            
        return self.__player_ratings.sort_values(by="Rating", ascending=False)

    def __calculate_match_score_accuracy(self, rating_table: pd.DataFrame):

        match_columns = self.matches.columns.to_list()
        winner_columns = [column for column in match_columns if column.startswith("Winner")]
        loser_columns = [column for column in match_columns if column.startswith("Loser")]

        def apply_match_score_accuracy(match):
            winners = match[winner_columns]
            winners = winners[winners != "---"]
            winner_rating = rating_table.loc[winners].sum()
            
            losers = match[loser_columns]
            losers = losers[losers != "---"]
            loser_rating = rating_table.loc[losers].sum()

            accuracy = pow(winner_rating / (winner_rating + loser_rating), match["K"])

            return accuracy

        return self.matches.apply(apply_match_score_accuracy, axis=1, result_type="expand")

    def __create_rating_table(self, step_size):
        
        ratings = []
        columns = []

        original_rating = self.__player_ratings["Rating"]

        def add_rating(rating, name):
            ratings.append(rating.apply(lambda x: pow(10, (x / 40))))
            columns.append(name)

        def add_alternative_rating(player, rating_change):
            alternative_rating = original_rating.copy()
            alternative_rating.loc[player] += rating_change
                
            add_rating(alternative_rating, (player, rating_change))

        add_rating(original_rating, MatchRating.original_rating_name)

        for player in self.players.index:
            add_alternative_rating(player, step_size)
            add_alternative_rating(player, -step_size)
            
        rating_table = pd.concat(ratings, axis=1)
        rating_table.columns = columns

        return rating_table

In [5]:
rating_system = MatchRating('Data/Players2.csv', 'Data/Matches2.csv')

rankings = rating_system.calculate_ratings(1000, 1024)
rankings = rating_system.calculate_ratings(1000, 512)
rankings = rating_system.calculate_ratings(1000, 256)
rankings = rating_system.calculate_ratings(1000, 128)
rankings = rating_system.calculate_ratings(1000, 64)
rankings = rating_system.calculate_ratings(1000, 32)
rankings = rating_system.calculate_ratings(1000, 16)
rankings = rating_system.calculate_ratings(1000, 8)
rankings = rating_system.calculate_ratings(1000, 4)
rankings = rating_system.calculate_ratings(1000, 2)
rankings = rating_system.calculate_ratings(1000, 1)

rankings2 = rankings - rankings.loc["AVG"] + 1000
rankings2.drop("AVG", inplace=True)
rankings2 = rankings2.sort_values(by="Rating", ascending=False)
rankings2

Step Size=1024, Iteration 1, Accuracy: 0.5
Step Size=512, Iteration 1, Accuracy: 0.5
Step Size=256, Iteration 1, Accuracy: 0.5
Step Size=128, Iteration 1, Accuracy: 0.50446
Step Size=128, Iteration 2, Accuracy: 0.50446
Step Size=64, Iteration 1, Accuracy: 0.51094
Step Size=64, Iteration 2, Accuracy: 0.51158
Step Size=64, Iteration 3, Accuracy: 0.51222
Step Size=64, Iteration 4, Accuracy: 0.51285
Step Size=64, Iteration 5, Accuracy: 0.51349
Step Size=64, Iteration 6, Accuracy: 0.51413
Step Size=64, Iteration 7, Accuracy: 0.51413
Step Size=32, Iteration 1, Accuracy: 0.51966
Step Size=32, Iteration 2, Accuracy: 0.52542
Step Size=32, Iteration 3, Accuracy: 0.53867
Step Size=32, Iteration 4, Accuracy: 0.55716
Step Size=32, Iteration 5, Accuracy: 0.56221
Step Size=32, Iteration 6, Accuracy: 0.56618
Step Size=32, Iteration 7, Accuracy: 0.56902
Step Size=32, Iteration 8, Accuracy: 0.57187
Step Size=32, Iteration 9, Accuracy: 0.5743
Step Size=32, Iteration 10, Accuracy: 0.57673
Step Size=32, It